In [1]:
import pandas as pd
import geopandas as gpd
import fiona

In [2]:
parcels_geo_pba50plus = pd.read_csv(r'M:\urban_modeling\baus\BAUS Inputs\basis_inputs\crosswalks\parcels_geography_2024_02_14.csv')
parcels_geo_pba50 = pd.read_csv(r'C:\Users\ywang\Box\Modeling and Surveys\Urban Modeling\Bay Area UrbanSim\PBA50\PBA50 Final Blueprint Large General Input Data\2021_01_12_parcels_geography.csv')

C:\Users\ywang\AppData\Local\Temp\ipykernel_18180\370113606.py:2: DtypeWarning: Columns (10,20) have mixed types. Specify dtype option on import or set low_memory=False.
  parcels_geo_pba50 = pd.read_csv(r'C:\Users\ywang\Box\Modeling and Surveys\Urban Modeling\Bay Area UrbanSim\PBA50\PBA50 Final Blueprint Large General Input Data\2021_01_12_parcels_geography.csv')


In [3]:
display(parcels_geo_pba50plus.head(3))
display(parcels_geo_pba50.head(3))

,PARCEL_ID,geom_id,jurisdiction_id,juris,ACRES,tpp_id,opp_id,perffoot,perfarea,urbanized,...,nodev,tra_id,gg_id,pda_id,hra_id,epc_id,ugb_id,ppa_50plus,ppa_id,exd_id
0,229116,10305106092872,41992,livermore,3.360520,NaN,NaN,1,0,1,...,0.0,NaN,NaN,NaN,NaN,NaN,UGB,NaN,NaN,NaN
1,244166,11107351665227,41992,livermore,1.294423,NaN,NaN,1,0,1,...,0.0,NaN,NaN,NaN,NaN,NaN,UGB,NaN,NaN,NaN
2,202378,11030175960628,33000,hayward,14.993605,NaN,NaN,1,0,0,...,0.0,NaN,NaN,NaN,NaN,NaN,UGB,NaN,NaN,exd


,PARCEL_ID,geom_id,jurisdiction_id,juris_name_full,juris_id,juris,ACRES,pda_id_pba40,tpp_id,exp_id,...,fbp_gg_id,pda_id_pba50_fb,fbp_tra_id,fbp_sesit_id,fbp_ppa_id,fbp_exp2020_id,fbp_exsfd_id,fbpzoningmodcat,fbpchcat,nodev
0,229116,10305106092872,41992,livermore,livr,livermore,3.360520,NaN,NaN,NaN,...,NaN,NaN,NaN,HRADIS,NaN,in,NaN,livermoreNANAHRADISNAinNA,NANAHRADISNAin,0.0
1,244166,11107351665227,41992,livermore,livr,livermore,1.294423,NaN,NaN,NaN,...,NaN,NaN,tra3,DIS,NaN,in,NaN,livermoreNAtra3DISNAinNA,NAtra3DISNAin,0.0
2,202378,11030175960628,33000,hayward,hayw,hayward,14.993605,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,in,NaN,haywardNANANANAinNA,NANANANAin,0.0


In [4]:
parcels_geo_pba50plus['ugb_id'].value_counts(dropna=False)

ugb_id
UGB          1669191
Uninc UGB     167059
NaN           119958
Name: count, dtype: int64

In [5]:
parcels_geo_pba50['fbp_exp2020_id'].value_counts(dropna=False)

fbp_exp2020_id
in           1669609
inun          149355
out           102370
exp1           13241
exp3_au         7904
exp1_np         5979
exp0            2970
expmax_np       2644
exp2_np         1017
expmax           695
ubz              322
exp0_np           94
ubz_np             8
Name: count, dtype: int64

In [6]:
# bring PBA50 `fbp_exp2020_id` into the new data
parcels_geo_pba50plus_NP = pd.merge(
    parcels_geo_pba50plus,
    parcels_geo_pba50[['PARCEL_ID', 'fbp_exp2020_id']],
    on='PARCEL_ID',
    how='left'
)
# check
print(parcels_geo_pba50plus_NP.shape[0])
print(parcels_geo_pba50plus_NP['PARCEL_ID'].nunique())

# assign UGB expansion categories for parcels outside of PBA50+ UGB definition
parcels_geo_pba50plus_NP['ugb_id_NP'] = parcels_geo_pba50plus_NP['ugb_id']
parcels_geo_pba50plus_NP.loc[
    ~parcels_geo_pba50plus_NP['ugb_id_NP'].isin(['UGB', 'Uninc UGB']) & \
    (parcels_geo_pba50plus_NP['fbp_exp2020_id'].isin(
        ['exp0', 'exp0_np', 'exp1', 'exp1_np', 'exp2_np', 'exp3_au', 'expmax_np'])), 'ugb_id_NP'] = parcels_geo_pba50plus_NP['fbp_exp2020_id']

# double check parcel counts
print(parcels_geo_pba50plus_NP.loc[~parcels_geo_pba50plus_NP['ugb_id'].isin(['UGB', 'Uninc UGB'])]['fbp_exp2020_id'].value_counts(dropna=False))

print(parcels_geo_pba50plus_NP['ugb_id_NP'].value_counts(dropna=False))

1956208
1956208
fbp_exp2020_id
out          90523
exp1         10180
exp3_au       7297
exp1_np       4879
expmax_np     2573
inun          1049
in             884
exp2_np        880
exp0           783
expmax         695
ubz            162
exp0_np         51
ubz_np           2
Name: count, dtype: int64
ugb_id_NP
UGB          1669191
Uninc UGB     167059
NaN            93315
exp1           10180
exp3_au         7297
exp1_np         4879
expmax_np       2573
exp2_np          880
exp0             783
exp0_np           51
Name: count, dtype: int64


In [7]:
# clean up and write out

parcels_geo_pba50plus_NP_clean = parcels_geo_pba50plus_NP.drop(['ugb_id', 'fbp_exp2020_id'], axis=1)
parcels_geo_pba50plus_NP_clean.rename(columns = {'ugb_id_NP': 'ugb_id'}, inplace=True)
print(list(parcels_geo_pba50plus_NP_clean))
print(parcels_geo_pba50plus_NP_clean.shape[0])
display(parcels_geo_pba50plus_NP_clean.head(3))

parcels_geo_pba50plus_NP_clean.to_csv(r'M:\urban_modeling\baus\BAUS Inputs\basis_inputs\crosswalks\parcels_geography_NP_2024_04_29.csv', index=False)


['PARCEL_ID', 'geom_id', 'jurisdiction_id', 'juris', 'ACRES', 'tpp_id', 'opp_id', 'perffoot', 'perfarea', 'urbanized', 'exp2020_id', 'exsfd_id', 'dis_id', 'nodev', 'tra_id', 'gg_id', 'pda_id', 'hra_id', 'epc_id', 'ppa_50plus', 'ppa_id', 'exd_id', 'ugb_id']
1956208


,PARCEL_ID,geom_id,jurisdiction_id,juris,ACRES,tpp_id,opp_id,perffoot,perfarea,urbanized,...,nodev,tra_id,gg_id,pda_id,hra_id,epc_id,ppa_50plus,ppa_id,exd_id,ugb_id
0,229116,10305106092872,41992,livermore,3.360520,NaN,NaN,1,0,1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UGB
1,244166,11107351665227,41992,livermore,1.294423,NaN,NaN,1,0,1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UGB
2,202378,11030175960628,33000,hayward,14.993605,NaN,NaN,1,0,0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,exd,UGB


In [8]:
set(parcels_geo_pba50plus) == set(parcels_geo_pba50plus_NP_clean)

True

In [9]:
## spatial files for QAQC

p10_dataset_path = r'M:\urban_modeling\data\basemap\parcels\Parcels 2010 Final PBA40 Version.gdb'
print(p10_dataset_path)
p10_layers = fiona.listlayers(p10_dataset_path)
print(p10_layers)

p10_gdf = gpd.read_file(p10_dataset_path, driver='fileGDB', layer='parcels2010')
print(p10_gdf.shape[0])
print(p10_gdf['PARCEL_ID'].nunique())

print(p10_gdf.crs)

M:\urban_modeling\data\basemap\parcels\Parcels 2010 Final PBA40 Version.gdb
['parcels2010']
1956208
1956208
epsg:26910


In [10]:
p10_ugb_comp = pd.merge(
    p10_gdf[['PARCEL_ID', 'geometry']],
    parcels_geo_pba50plus_NP[['PARCEL_ID', 'juris', 'ugb_id', 'fbp_exp2020_id']],
    on='PARCEL_ID',
    how='outer'
)

display(p10_ugb_comp.head())
print(p10_ugb_comp.shape[0])
print(type(p10_ugb_comp))

# p10_ugb_comp.to_file(r'M:\urban_modeling\baus\PBA50Plus\investigation\growth_patterns\qaqc_pba50plus_DBP_growth_patterns\ugb_comparison.shp')

,PARCEL_ID,geometry,juris,ugb_id,fbp_exp2020_id
0,229116.0,"MULTIPOLYGON (((605993.404 4168316.032, 605993...",livermore,UGB,in
1,244166.0,"MULTIPOLYGON (((613491.065 4175262.642, 613490...",livermore,UGB,in
2,202378.0,"MULTIPOLYGON (((586745.708 4168287.089, 586746...",hayward,UGB,in
3,2004420.0,"MULTIPOLYGON (((520226.869 4286274.188, 520222...",unincorporated_sonoma,NaN,out
4,340332.0,"MULTIPOLYGON (((590655.279 4155999.718, 590590...",fremont,UGB,in


1956208
<class 'geopandas.geodataframe.GeoDataFrame'>


In [11]:
p10_ugb_NP = pd.merge(
    p10_gdf[['PARCEL_ID', 'geometry']],
    parcels_geo_pba50plus_NP_clean[['PARCEL_ID', 'juris', 'ugb_id']],
    on='PARCEL_ID',
    how='outer'
)

display(p10_ugb_NP.head())
print(p10_ugb_NP.shape[0])
print(type(p10_ugb_NP))

p10_ugb_NP = p10_ugb_NP.loc[p10_ugb_NP['ugb_id'].notnull()]

p10_ugb_NP.to_file(r'M:\urban_modeling\baus\PBA50Plus\investigation\growth_patterns\qaqc_pba50plus_DBP_growth_patterns\ugb_NP_version_20240429.shp')

,PARCEL_ID,geometry,juris,ugb_id
0,229116.0,"MULTIPOLYGON (((605993.404 4168316.032, 605993...",livermore,UGB
1,244166.0,"MULTIPOLYGON (((613491.065 4175262.642, 613490...",livermore,UGB
2,202378.0,"MULTIPOLYGON (((586745.708 4168287.089, 586746...",hayward,UGB
3,2004420.0,"MULTIPOLYGON (((520226.869 4286274.188, 520222...",unincorporated_sonoma,NaN
4,340332.0,"MULTIPOLYGON (((590655.279 4155999.718, 590590...",fremont,UGB


1956208
<class 'geopandas.geodataframe.GeoDataFrame'>


In [12]:
p10_ugb_NP.shape[0]

1862893